In [ ]:
# Load helper code
!git clone https://github.com/edgi-govdata-archiving/ECHO_modules.git &>/dev/null;
import pandas
import ipywidgets as widgets

In [ ]:
# Make selections about programs, active facilities, and NAICS codes
programs = widgets.SelectMultiple(
    options=['NPDES_FLAG', 'AIR_FLAG', 'RCRA_FLAG'],
    description='Programs',
    disabled=False
)
active = widgets.Checkbox(
    value=False,
    description='Only active facilities?',
    disabled=False
)
naics = widgets.Text(
    value='',
    placeholder='Enter NAICS codes here, separated with a comma', #For example: 2111,213111,213112
    description='NAICS codes',
    disabled=False
)
states = widgets.Dropdown(
    options=['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY'],
    value='CA',
    description='State:',
    disabled=False,
)
display(programs, active, naics, states)

SelectMultiple(description='Programs', options=('NPDES_FLAG', 'AIR_FLAG', 'RCRA_FLAG'), value=())

Checkbox(value=False, description='Only active facilities?')

Text(value='', description='NAICS codes', placeholder='Enter NAICS codes here, separated with a comma')

Dropdown(description='State:', index=4, options=('AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', '…

In [ ]:
!pip install geopandas  &>/dev/null;
!pip install topojson &>/dev/null;
!pip install rtree &>/dev/null;

import geopandas
import shapely
import topojson
import rtree
import folium

**get census block data:**

get_echo_data or get_data ??

In [ ]:
#sql = ‘select * from “EJSCREEN_2021_USPR” where “ST_ABBREV” = selected_state’
#cbgs = get_data(sql)
from ECHO_modules.get_data import get_echo_data
sql =  'select * from “EJSCREEN_2021_USPR” where “ST_ABBREV” = \‘’+state_widget.value+‘\’'
cbgs = get_echo_data(sql) # get_echo_data or get_data ??
cbgs

HTTPError: ignored

In [ ]:
# Create the SQL query string
# Filter by programs
sql_string = '(('
pgms = ''
if len(programs.value) > 0:
  for pgm in programs.value:
    pgms += "\""+pgm+"\" = \'Y\' or "
pgms = pgms[:-4]
sql_string += pgms

# Filter by active
if active.value:
  sql_string += ' and "FAC_ACTIVE_FLAG" = \'Y\')'
else:
  sql_string += ')'

# Filter by NAICS
pgm_key = {"NPDES_FLAG": "CWA_NAICS", "AIR_FLAG": "CAA_NAICS", "RCRA_FLAG": "RCRA_NAICS"}
naics_string = ' and ('
for n in naics.value.replace(" ", "").split(","):
    if len(n) == 6:
      naics_string += '"FAC_NAICS_CODES" like \'%'+n+'%\' or '
      for pgm in programs.value:
        naics_string += '"'+pgm_key[pgm]+'" like \'%'+n+'%\' or '
    else:
      missing = 6 - len(n)
      wilds = "%" * missing
      naics_string += '"FAC_NAICS_CODES" like \' %'+n+''+wilds+'\' or '
      naics_string += '"FAC_NAICS_CODES" like \''+n+''+wilds+'\' or '
      for pgm in programs.value:
        naics_string += '"'+pgm_key[pgm]+'" like \' %'+n+''+wilds+'\' or '
        naics_string += '"'+pgm_key[pgm]+'" like \''+n+''+wilds+'\' or '

# String together
naics_string = naics_string[:-4]
naics_string += "))"
sql_string = sql_string + naics_string
print(sql_string)

(() and ("FAC_NAICS_CODES" like ' %%%%%%%' or "FAC_NAICS_CODES" like '%%%%%%'))


In [ ]:
# Access the ECHO_EXPORTER table
# Output will be the matching tracked by EPA
from ECHO_modules.get_data import get_echo_data

sql = 'select "FAC_NAME", "CWA_NAICS", "CAA_NAICS", "RCRA_NAICS", "FAC_NAICS_CODES", "FAC_LAT", "FAC_LONG", "FAC_PERCENT_MINORITY", "FAC_TOTAL_PENALTIES", "FAC_QTRS_WITH_NC", "FAC_PENALTY_COUNT", "EJSCREEN_FLAG_US" from "ECHO_EXPORTER" where ' + sql_string
try:
  echo = get_echo_data( sql )
except pandas.errors.EmptyDataError:
  print("\nThere are no records.\n")
  
echo

HTTPError: ignored

In [ ]:
print(type(echo))
points = geopandas.GeoDataFrame(echo, geometry=geopandas.points_from_xy(echo.FAC_LONG, echo.FAC_LAT), crs = 4269)
print(type(points))
points.head()


NameError: ignored

In [ ]:
#high level: create a 2d array/df with rows and cols as facility ids and census blocks. 1/0 if census is in 5 mi range of facility. 
#will need helper function to get census blocks in 5 mi range of specific facility; then can update df from here. 

echo_proj = echo.to_crs({"init": "EPSG:3857"})
cbgs_proj = cbgs.to_crs({"init": "EPSG:3857"})

# so 5 miles would be 8046.7 meters

x = cbgs_proj.buffer(8047).unary_union

neighbours = echo_proj["geometry"].intersection(x)

# print all the nearby points
print(echo_proj[~neighbours.is_empty])

In [ ]:
# Export the data to the "Files" side panel for download
echo.to_csv("data.csv")

In [ ]:
# Map the facilities
import folium
from folium.plugins import MarkerCluster
import math

map = folium.Map()
marker_cluster = MarkerCluster().add_to(map)

for index, row in echo.iterrows():
  if math.isnan(row["FAC_LAT"]) or math.isnan(row["FAC_LONG"]): #ignore facilities with no spatial data
    pass
  else:
    marker = folium.Marker(
        location = [float(row["FAC_LAT"]), float(row["FAC_LONG"])],
        popup = row["FAC_NAME"]
    ).add_to(marker_cluster)

map